## Runs DE on different clusters. Also runs DE within each cluster for each condition

In [1]:
# Input info
# integrated_f = "/data/isshamie/mito_lineage/output/annotation/TcellDupi_may17_2021/MTblacklist/mergedSamples/allSamples.integrated.rds"
# outdir = "/data/isshamie/mito_lineage/output/annotation/TcellDupi_may17_2021/MTblacklist/mergedSamples/DE" 
integrated_f = "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/allSamples.integrated.rds"
sample_names = "preB,postB"
outdir = "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/DE_minPct_01"

# Parameters
nTop = 25000
assay="RNA"

minPct = 0.01
cores = 4


comps_f = ""
#cond.comparisons <- "A,preA,postA;B,preB,postB"

In [2]:
outdir

[1] "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/DE_minPct_01"

In [3]:
clust_outdir = file.path(outdir, "clusters")
clust_atac_outdir = file.path(outdir, "clusters_atac")
cond_outdir = file.path(outdir, "conditions_clusters")
cons_outdir = file.path(outdir, "conditions_conserved")
dir.create(outdir, showWarnings = FALSE)
dir.create(clust_outdir, showWarnings = FALSE)
dir.create(cond_outdir, showWarnings = FALSE)
dir.create(cons_outdir, showWarnings = FALSE)
dir.create(clust_atac_outdir, showWarnings = FALSE)

In [4]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
#samples <- unlist(strsplit(samples, ","))

sample_names

[1] "preB"  "postB"

In [5]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")

library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Attaching SeuratObject

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required pa

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [6]:
integrated <- readRDS(integrated_f)#file.path(indir, paste0("allSamples.integrated.rds")))
integrated

An object of class Seurat 
62328 features across 3683 samples within 2 assays 
Active assay: RNA (20010 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [7]:
integrated[['peak_region_fragments']]

,peak_region_fragments
,<dbl>
preB_AAACGAAGTCACTCTC-1,5325
preB_AAACGAAGTCCCTAAA-1,5461
preB_AAACGAAGTCTGTGTA-1,1246
preB_AAACTCGCAACTCGTA-1,7857
preB_AAACTCGCAGAACGAC-1,8179
preB_AAACTGCCACACATGT-1,1672
preB_AAAGATGGTTTGACCA-1,4167
preB_AAAGATGTCCGTCAAA-1,2221
preB_AAAGATGTCTCTGTTA-1,4294


In [8]:
integrated[[]]

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,⋯,TSS.enrichment,TSS.percentile,pct_reads_in_peaks,blacklist_ratio,high.tss,nucleosome_group,ATAC_snn_res.0.8,seurat_clusters,nCount_RNA,nFeature_RNA
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<fct>,<dbl>,<int>
preB_AAACGAAGTCACTCTC-1,preB,3754,3754,38954,29912,335,145,1057,0,7505,⋯,2.104279,0.48,70.95270,0,High,NS < 4,0,0,3751,2539
preB_AAACGAAGTCCCTAAA-1,preB,4130,4130,24309,15056,258,113,968,0,7914,⋯,1.425693,0.06,69.00430,0,Low,NS < 4,6,6,4280,2832
preB_AAACGAAGTCTGTGTA-1,preB,683,683,17674,15525,189,72,200,0,1688,⋯,2.277722,0.61,73.81517,0,High,NS < 4,2,2,581,517
preB_AAACTCGCAACTCGTA-1,preB,5270,5270,39016,26386,437,223,1941,0,10029,⋯,2.793040,0.82,78.34281,0,High,NS < 4,6,6,5460,3063
preB_AAACTCGCAGAACGAC-1,preB,5575,5575,70990,51473,759,331,2599,0,15828,⋯,1.793144,0.25,51.67425,0,Low,NS < 4,5,5,8266,3971
preB_AAACTGCCACACATGT-1,preB,968,968,23591,20903,197,96,328,0,2067,⋯,2.353202,0.66,80.89018,0,High,NS < 4,0,0,792,645
preB_AAAGATGGTTTGACCA-1,preB,2719,2719,52480,41104,526,355,1808,0,8687,⋯,1.327561,0.03,47.96823,0,Low,NS < 4,5,5,3984,2639
preB_AAAGATGTCCGTCAAA-1,preB,1677,1677,13817,9880,167,66,514,0,3190,⋯,1.923077,0.35,69.62382,0,Low,NS < 4,7,7,1573,1244
preB_AAAGATGTCTCTGTTA-1,preB,3013,3013,28453,20833,270,144,1272,0,5934,⋯,2.449402,0.72,72.36266,0,High,NS < 4,9,9,2832,1951


## Compute DE for each cluster

In [9]:
plotDE <- function(integrated, de.results, i, outdir){
    try
    {
    plot1 <- VlnPlot(
      object = integrated,
      features = rownames(de.results)[1],
      pt.size = 0.1,
      idents = i
    )
    plot2 <- FeaturePlot(
      object = integrated,
      features = rownames(de.results)[1],
      pt.size = 0.1
    )

    plot3 <- FeaturePlot(
      object = integrated,
      features = rownames(de.results)[2],
      pt.size = 0.1
    )
    
    plot1 | plot2 | plot3
    
    ggsave(file.path(outdir,paste0("cluster_",i,".DE.GeneActivity.top2.png")))
    
    return(c(plot1, plot2, plot3))
    }
    return
}

## Loop through each cluster and run DA for RNA-seq

In [ ]:

cluster.ids <- sort(unique(integrated$seurat_clusters))
for (i in cluster.ids){ #or however many clusters you have
try({
    print(paste("cluster", i))
    da.peaks <- FindMarkers(
                  object = integrated,
                  ident.1 = i, #"CD4 Naive",
                  min.pct = minPct,
                  test.use = 'LR', latent.vars = 'peak_region_fragments'
                )
    print(file.path(clust_outdir, paste0("cluster_",i,".DE.GeneActivity.csv")))
    print(head(da.peaks))
    write.csv(da.peaks, file=file.path(clust_outdir, paste0("cluster_",i,".DE.GeneActivity.csv")))
    print(paste('saved cluster', i))
    plotDE(integrated, da.peaks, i, clust_outdir)
    print(paste('plotted cluster', i))
    
})
}

[1] "cluster 0"
[1] "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/DE_minPct_01/clusters/cluster_0.DE.GeneActivity.csv"
               p_val avg_log2FC pct.1 pct.2    p_val_adj
ANKRD44 1.608616e-47  0.5259003 0.628 0.419 3.218840e-43
PLCG2   2.319839e-44 -0.5220017 0.114 0.392 4.641999e-40
TVP23A  1.379250e-42 -0.3498266 0.034 0.274 2.759880e-38
LPCAT1  2.537309e-40 -0.4206341 0.086 0.341 5.077155e-36
ADAM9   3.522462e-37 -0.4613041 0.244 0.533 7.048447e-33
PFDN1   2.366108e-36 -0.4337015 0.144 0.420 4.734583e-32
[1] "saved cluster 0"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 0"
[1] "cluster 1"
[1] "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/DE_minPct_01/clusters/cluster_1.DE.GeneActivity.csv"
               p_val avg_log2FC pct.1 pct.2     p_val_adj
MAPK1  3.886688e-153  0.7392848 0.593 0.090 7.777264e-149
CDC123 2.019547e-147  0.7174858 0.593 0.092 4.041115e-143
TVP23A 1.521371e-142  0.7688801 0.673 0.141 3.044263e-138
BANP   3.648529e-142  0.7585823 0.592 0.112 7.300707e-138
PLCG2  1.883437e-134  0.9121822 0.769 0.253 3.768757e-130
PAX5   3.807500e-130  0.8270445 0.692 0.208 7.618807e-126
[1] "saved cluster 1"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 1"
[1] "cluster 2"
[1] "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/DE_minPct_01/clusters/cluster_2.DE.GeneActivity.csv"
                    p_val avg_log2FC pct.1 pct.2    p_val_adj
GNG7         5.394666e-17 -0.3750258 0.173 0.497 1.079473e-12
TVP23A       1.350391e-16 -0.2974379 0.050 0.250 2.702133e-12
RPTOR        1.830396e-15 -0.3137415 0.490 0.805 3.662622e-11
RELA         2.236225e-14 -0.2861981 0.173 0.480 4.474686e-10
TMEM91       1.145537e-13 -0.3183283 0.217 0.527 2.292219e-09
CTC-435M10.3 2.198941e-13 -0.3091857 0.269 0.585 4.400081e-09
[1] "saved cluster 2"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 2"
[1] "cluster 3"
[1] "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/DE_minPct_01/clusters/cluster_3.DE.GeneActivity.csv"
                     p_val avg_log2FC pct.1 pct.2     p_val_adj
RNF103-CHMP3 1.029231e-113  0.9171994 0.752 0.285 2.059491e-109
CHMP3        1.185500e-113  0.9169298 0.752 0.286 2.372186e-109
KLRC4-KLRK1   2.030386e-57  0.4369801 0.391 0.084  4.062802e-53
DKK2          2.494363e-35  0.4533115 0.907 0.706  4.991221e-31
ANKRD6        1.923408e-32  0.4294474 0.885 0.680  3.848740e-28
RHOH          8.688419e-25  0.3733322 0.728 0.491  1.738553e-20
[1] "saved cluster 3"


Saving 6.67 x 6.67 in image



[1] "plotted cluster 3"
[1] "cluster 4"
[1] "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/DE_minPct_01/clusters/cluster_4.DE.GeneActivity.csv"
                p_val avg_log2FC pct.1 pct.2    p_val_adj
C10ORF68 4.693034e-43  0.6324593 0.807 0.557 9.390760e-39
ATXN1    4.193059e-36  0.5520842 0.904 0.703 8.390310e-32
MKLN1    1.047376e-30  0.5201333 0.938 0.821 2.095800e-26
NBEAL1   1.531327e-29  0.5180847 0.592 0.317 3.064185e-25
BRIP1    1.514170e-25  0.4335581 0.765 0.513 3.029854e-21
FAM129A  3.988469e-21  0.4290504 0.490 0.272 7.980926e-17
[1] "saved cluster 4"


Saving 6.67 x 6.67 in image



In [ ]:
library(stats)
p.adjust(p, method = p.adjust.methods, n = length(p))
p.adjust.methods
# c("holm", "hochberg", "hommel", "bonferroni", "BH", "BY",
#   "fdr", "none")

## Loop through each cluster and run DA for ATAC-seq

In [ ]:
DefaultAssay(integrated) <- "ATAC"
cluster.ids <- sort(unique(integrated$seurat_clusters))
for (i in cluster.ids){ #or however many clusters you have
try({
    print(paste("cluster", i))
    da.peaks <- FindMarkers(
                  object = integrated,
                  ident.1 = i, #"CD4 Naive",
                  min.pct = minPct,
                  test.use = 'LR', latent.vars = 'peak_region_fragments'
                )
    print(head(da.peaks))
    write.csv(da.peaks, file=file.path(clust_atac_outdir, paste0("cluster_",i,".DE.PeakActivity.csv")))
    print(paste('saved cluster', i))
    plotDE(integrated, da.peaks, i, clust_atac_outdir)
    print(paste('plotted cluster', i))
    
})
}

# put back to RNA
DefaultAssay(integrated) <- "RNA"


## Stimulus specific response

In [ ]:
stimout <- function(outdir, sample_names){
    cluster.ids <- sort(unique(integrated$seurat_clusters))
    integrated$celltype.stim <- paste(integrated$seurat_clusters, integrated$orig.ident, sep = "_")
    integrated$celltype <- integrated$seurat_clusters
    Idents(integrated) <- "celltype.stim"

    for (c in cluster.ids){
        try({
            response <- FindMarkers(integrated, 
                                    ident.1 = paste0(c, "_", sample_names[[1]]), 
                                    ident.2 = paste0(c, "_", sample_names[[2]]), 
                                    verbose = FALSE,
                                    test.use = 'LR', min.pct = minPct,
                                    latent.vars = 'peak_region_fragments'
                                   )
            if (!(dim(response)[1]==0)){

                print(head(response, n = 15))
                curr_clust <- subset(integrated, seurat_clusters == c)
                avg_curr_clust <- data.frame(log1p(AverageExpression(curr_clust, verbose = FALSE)$RNA))
                avg_curr_clust$gene <- rownames(avg_curr_clust)

                p1 <- ggplot(avg_curr_clust, aes_string(paste0("X", c, "_", sample_names[[1]]), paste0("X", c, "_", sample_names[[2]]))) + geom_point() + ggtitle(paste("Cluster", c))
                p1 <- LabelPoints(plot = p1, points = rownames(head(response, n = 15)), repel = TRUE)
                plot_grid(p1)
                write.csv(response, file=file.path(outdir,paste0("cluster_",i,".conditionDE.csv")))
                ggsave(file.path(outdir,paste0("cluster_",i,".conditionScatter.png")))

            }
        })

    }
}



In [ ]:
if ((comps_f=="") | comps_f == "NULL"){
    stimout(cond_outdir, sample_names)

}else{
    comps <- unlist(strsplit(comps_f, ";")[[1]])
    comps <- lapply(comps, function(x) unlist(strsplit(x, ',')[[1]]))
    for (i in comps){
        print(i)
        curr_sample_names <- i[2:3]
        curr_cond_outdir <- file.path(cond_outdir, i[1])
        dir.create(curr_cond_outdir, showWarnings = FALSE)
        stimout(curr_cond_outdir, curr_sample_names)
    }
}

# comps_f <- "A,preA,postA"#;B,preB,postB"
# comps <- unlist(strsplit(comps_f, ";")[[1]])
# #samples <- unlist(strsplit(samples, ","))

# comps <- lapply(comps, function(x) unlist(strsplit(x, ',')[[1]]))
# comps
# for (i in comps){
#     print(i)
#     sample_names <- i[2:3]
# }
# sample_names


                    
# cluster.ids <- sort(unique(integrated$seurat_clusters))
# integrated$celltype.stim <- paste(integrated$seurat_clusters, integrated$orig.ident, sep = "_")
# integrated$celltype <- integrated$seurat_clusters
# Idents(integrated) <- "celltype.stim"

# for (c in cluster.ids){
#     try({
#         response <- FindMarkers(integrated, 
#                                 ident.1 = paste0(c, "_", sample_names[[1]]), 
#                                 ident.2 = paste0(c, "_", sample_names[[2]]), 
#                                 verbose = FALSE,
#                                 test.use = 'LR', min.pct = 0.1,
#                                 latent.vars = 'peak_region_fragments'
#                                )
#         if (!(dim(response)[1]==0)){

#             print(head(response, n = 15))
#             curr_clust <- subset(integrated, seurat_clusters == c)
#             avg_curr_clust <- data.frame(log1p(AverageExpression(curr_clust, verbose = FALSE)$RNA))
#             avg_curr_clust$gene <- rownames(avg_curr_clust)

#             p1 <- ggplot(avg_curr_clust, aes_string(paste0("X", c, "_", sample_names[[1]]), paste0("X", c, "_", sample_names[[2]]))) + geom_point() + ggtitle(paste("Cluster", c))
#             p1 <- LabelPoints(plot = p1, points = rownames(head(response, n = 15)), repel = TRUE)
#             plot_grid(p1)
#             write.csv(response, file=file.path(cond_outdir,paste0("cluster_",i,".conditionDE.csv")))
#             ggsave(file.path(cond_outdir,paste0("cluster_",i,".conditionScatter.png")))

#         }
#     })

# }

## Gene Markers plot

In [ ]:
clrs =  brewer.pal(n = length(sample_names), name = "RdBu")

### Immune markers taken from Seurat

In [ ]:
Idents(integrated) <- "seurat_clusters"

In [ ]:
FeaturePlot(integrated, features = c("CD3D", "SELL", "CREM", "CD8A", "GNLY", "CD79A", "FCGR3A", 
    "CCL2", "PPBP"), min.cutoff = "q9")

ggsave(file=file.path(outdir,"seuratImmuneEmbed.png"))


In [ ]:
markers.to.plot <- c("CD3D", "CREM", "HSPH1", "SELL", "GIMAP5", "CACYBP", "GNLY", "NKG7", "CCL5", 
    "CD8A", "MS4A1", "CD79A", "MIR155HG", "NME1", "FCGR3A", "VMO1", "CCL2", "S100A9", "HLA-DQA1", 
    "GPR183", "PPBP", "GNG11", "HBA2", "HBB", "TSPAN13", "IL3RA", "IGJ")
DotPlot(integrated, features = rev(markers.to.plot), dot.scale = 8, cols=clrs,  
    split.by = "orig.ident") + RotatedAxis()
ggsave(file=file.path(outdir,"seuratImmuneDotPlot.png"))

### Immune markers taken from Dawn Lin et al Flt3l paper

In [ ]:
immune.markers = c('KIT',
                   'LY6E',
                   'ITGAX',
                   'SLAMF1',
                   'CD34',
                   'FCGR3A',
                   'PTPRC',
                   'SLAMF2',
                   'IL7R',
                   'ITGAM')

DotPlot(integrated, features = rev(immune.markers), dot.scale = 8, 
        cols = clrs,
    split.by = "orig.ident") + RotatedAxis()
ggsave(file=file.path(outdir,"linImmuneDotPlot.png"))

In [ ]:
FeaturePlot(integrated, features = immune.markers, min.cutoff = "q9")
ggsave(file=file.path(outdir,"linImmuneEmbed.png"))

## Conserved markers to help identify cell type

In [ ]:
topMarkers = c()
for (i in cluster.ids){ #or however many clusters you have
try({
    print(paste("cluster", i))
    cons.markers <- FindConservedMarkers(integrated, ident.1 = i, 
                                         grouping.var = "orig.ident", verbose = TRUE)

    write.csv(cons.markers, file=file.path(cons_outdir,paste0("cluster_",i,".conservedOverStim.GeneActivity.csv")))
    print(paste('saved cluster', i))
    print(head(cons.markers))
    topMarkers = c(topMarkers, rownames(cons.markers)[1])
    print(topMarkers)
})
}

head(cons.markers)

In [ ]:
sessionInfo()